 
## Overview of Filtering
Let us understand few important details related to filtering before we get into the solution
* Filtering can be done either by using `filter` or `>where`. These are like synonyms to each other.
* When it comes to the condition, we can either pass it in **SQL Style** or **Data Frame Style**.
* Example for SQL Style - `airlines.filter("IsArrDelayed = 'YES'").show() or airlines.where("IsArrDelayed = 'YES'").show()`
* Example for Data Frame Style - `airlines.filter(airlines["IsArrDelayed"] == 'YES').show()</mark> or <mark>airlines.filter(airlines.IsArrDelayed == 'YES').show()`. We can also use where instead of filter.
* Here are the other operations we can perform to filter the data - `!=`, `>`, `<`>, `>=`, `<=`, `LIKE`, `BETWEEN` with `AND`
* If we have to validate against multiple columns then we need to use boolean operations such as `AND` and `OR`.
* If we have to compare each column value with multiple values then we can use the `IN` operator.
    

Let us start spark context for this Notebook so that we can execute the code provided. You can sign up for our [10 node state of the art cluster/labs](https://labs.itversity.com/plans) to learn Spark SQL using our unique integrated LMS.

In [ ]:
from pyspark.sql import SparkSession

import getpass
username = getpass.getuser()

spark = SparkSession. \
    builder. \
    config('spark.ui.port', '0'). \
    enableHiveSupport. \
    appName(f'{username} | Python - Basic Transformations'). \
    master('yarn'). \
    getOrCreate()

If you are going to use CLIs, you can use Spark SQL using one of the 3 approaches.

**Using Spark SQL**

```
spark2-sql \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

**Using Scala**

```
spark2-shell \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

**Using Pyspark**

```
pyspark2 \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

### Tasks

Let us perform some tasks to understand filtering in detail. Solve all the problems by passing  conditions using both SQL Style as well as API Style.

* Read the data for the month of 2008 January.

In [ ]:
airlines_path = "/public/airlines_all/airlines-part/flightmonth=200801"

In [ ]:
airlines = spark. \
    read. \
    parquet(airlines_path)

In [ ]:
airlines.printSchema()

* Get count of flights which are departed late at origin and reach destination early or on time.


In [ ]:
airlines. \
    filter("IsDepDelayed = 'YES' AND IsArrDelayed = 'NO'"). \
    count()

* API Style

In [ ]:
from pyspark.sql.functions import col

In [ ]:
airlines. \
    filter((col("IsDepDelayed") == "YES") & 
           (col("IsArrDelayed") == "NO")
          ). \
    count()

In [ ]:
airlines. \
    filter((airlines["IsDepDelayed"] == "YES") & 
           (airlines.IsArrDelayed == "NO")
          ). \
    count()

* Get count of flights which are departed late from origin by more than 60 minutes.


In [ ]:
airlines. \
    filter("DepDelay > 60"). \
    count()


* API Style

In [ ]:
from pyspark.sql.functions import col

airlines. \
    filter(col("DepDelay") > 60). \
    count()

* Get count of flights which are departed early or on time but arrive late by at least 15 minutes.


In [ ]:
airlines. \
    filter("IsDepDelayed = 'NO' AND ArrDelay >= 15"). \
    count()

* API Style

In [ ]:
from pyspark.sql.functions import col

airlines. \
    filter((col("IsDepDelayed") == "NO") & (col("ArrDelay") >= 15)). \
    count()

* Get count of flights departed from following major airports - ORD, DFW, ATL, LAX, SFO.

In [ ]:
airlines. \
    filter("Origin IN ('ORD', 'DFW', 'ATL', 'LAX', 'SFO')"). \
    count()

In [ ]:
airlines.count()

* API Style

In [ ]:
from pyspark.sql.functions import col
c = col('x')
help(c.isin)

In [ ]:
from pyspark.sql.functions import col

airlines. \
    filter(col("Origin").isin("ORD", "DFW", "ATL", "LAX", "SFO")). \
    count()

* Add a column FlightDate by using Year, Month and DayOfMonth. Format should be **yyyyMMdd**.


In [ ]:
from pyspark.sql.functions import col, concat, lpad

airlines. \
    withColumn("FlightDate",
                concat(col("Year"),
                       lpad(col("Month"), 2, "0"),
                       lpad(col("DayOfMonth"), 2, "0")
                      )
              ). \
    show()

* Get count of flights departed late between 2008 January 1st to January 9th using FlightDate.


In [ ]:
from pyspark.sql.functions import col, concat, lpad

airlines. \
    withColumn("FlightDate",
               concat(col("Year"),
                      lpad(col("Month"), 2, "0"),
                      lpad(col("DayOfMonth"), 2, "0")
                     )
              ). \
    filter("IsDepDelayed = 'YES' AND FlightDate LIKE '2008010%'"). \
    count()

In [ ]:
from pyspark.sql.functions import col, concat, lpad

airlines. \
    withColumn("FlightDate",
               concat(col("Year"),
                      lpad(col("Month"), 2, "0"),
                      lpad(col("DayOfMonth"), 2, "0")
                     )
              ). \
    filter("""
           IsDepDelayed = 'YES' AND 
           FlightDate BETWEEN 20080101 AND 20080109
          """). \
    count()

* API Style

In [ ]:
from pyspark.sql.functions import col
c = col('x')
help(c.like)

In [ ]:
from pyspark.sql.functions import col, concat, lpad

airlines. \
    withColumn("FlightDate",
               concat(col("Year"),
                      lpad(col("Month"), 2, "0"),
                      lpad(col("DayOfMonth"), 2, "0")
                     )
              ). \
    filter((col("IsDepDelayed") == "YES") & 
           (col("FlightDate").like("2008010%"))
          ). \
    count()

In [ ]:
from pyspark.sql.functions import col
c = col('x')
help(c.between)

In [ ]:
from pyspark.sql.functions import col, concat, lpad

airlines. \
    withColumn("FlightDate",
               concat(col("Year"),
                      lpad(col("Month"), 2, "0"),
                      lpad(col("DayOfMonth"), 2, "0")
                     )
              ). \
    filter((col("IsDepDelayed") == "YES") & 
           (col("FlightDate").between("20080101", "20080109"))
          ). \
    count()

* Get number of flights departed late on Sundays.

In [ ]:
l = [('X',)]
df = spark.createDataFrame(l, "dummy STRING")

In [ ]:
from pyspark.sql.functions import current_date
df.select(current_date()).show()

In [ ]:
from pyspark.sql.functions import date_format

df.select(current_date(), date_format(current_date(), 'EE')).show()

In [ ]:
from pyspark.sql.functions import col, concat, lpad

airlines. \
    withColumn("FlightDate",
               concat(col("Year"),
                      lpad(col("Month"), 2, "0"),
                      lpad(col("DayOfMonth"), 2, "0")
                     )
              ). \
    filter("""
           IsDepDelayed = 'YES' AND 
           date_format(to_date(FlightDate, 'yyyyMMdd'), 'EEEE') = 'Sunday'
           """). \
    count()

* API Style

In [ ]:
from pyspark.sql.functions import col, concat, lpad, date_format, to_date

airlines. \
    withColumn("FlightDate",
               concat(col("Year"),
                      lpad(col("Month"), 2, "0"),
                      lpad(col("DayOfMonth"), 2, "0")
                     )
              ). \
    filter((col("IsDepDelayed") == "YES") &
           (date_format(
               to_date("FlightDate", "yyyyMMdd"), "EEEE"
           ) == "Sunday")
          ). \
    count()